# Generate pressure coefficients

Read config file

In [1]:
from cfdmod.use_cases.pressure.cp_config import CpConfig
import pathlib

cfg_path = pathlib.Path("./fixtures/tests/pressure/cp_params.yaml") 
cfg = CpConfig.from_file(cfg_path)

cfg

CpConfig(timestep_range=(10000.0, 40000.0), reference_pressure='average', U_H=10.0, statistics=['min', 'max', 'std', 'avg', 'skewness', 'kurtosis'])

Read Mesh